# Cuaderno de Notas de Métodos Numéricos
## Interpolación: Diferencias divididas e interpolación de Hermite.
### Prof. Jorge I. Zuluaga


#### Ejecutarme siempre

A continuación descargamos en el espacio virtual de este cuaderno todos los archivos necesarios para que las notas del curso funcionen bien:

In [ ]:
!git clone https://github.com/JorgeZuluaga/NotasMetodosNumericos
!ln -s NotasMetodosNumericos mn
!make -C mn pull

Cloning into 'NotasMetodosNumericos'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 108 (delta 46), reused 82 (delta 20), pack-reused 0
Receiving objects: 100% (108/108), 8.53 MiB | 19.32 MiB/s, done.
Resolving deltas: 100% (46/46), done.
make: Entering directory '/content/NotasMetodosNumericos'
git reset --hard HEAD
HEAD is now at 876de31 Commit
git pull origin master
From https://github.com/JorgeZuluaga/NotasMetodosNumericos
 * branch            master     -> FETCH_HEAD
Already up to date.
make: Leaving directory '/content/NotasMetodosNumericos'


## Un vistazo de la clase

  Qué veremos en esta clase:

  - Diferencias divididas
  - Interpolación de Hermite

  Para leer en el libro: sección 3.3 "Diferencias divididas" y sección 3.4 "Interpolación de Hermite"

## Diferencias divididas

- El método de las diferencias divididas aparece cuando queremos escribir el Polinomio interpolante en la forma:

  $$
P_{n}(x)=a_{0}+a_{1}\left(x-x_{0}\right)+a_{2}\left(x-x_{0}\right)\left(x-x_{1}\right)+\cdots+a_{n}\left(x-x_{0}\right) \cdots\left(x-x_{n-1}\right)
$$

- Nótese que por la condición de que $P_n(x_i)=f(x_i)$ entonces:
  
  $$
  a_0=f(x_0)
  $$
  y 
  $$
  a_{1}=\frac{f\left(x_{1}\right)-f\left(x_{0}\right)}{x_{1}-x_{0}}
  $$

- Si definimos:

  $$
  f\left[x_{i}\right]=f\left(x_{i}\right)
  $$
  
  y 

  $$
  f\left[x_{i}, x_{i+1}\right]=\frac{f\left[x_{i+1}\right]-f\left[x_{i}\right]}{x_{i+1}-x_{i}}
  $$

  entonces:

  $$
  a_0=f[x_0]
  $$

  y 

  $$
  a_1=f[x_0,x_1]
  $$

- Llamamos a $f[x_i]$ y a $f[x_i,x_{i+1}]$ diferencias divididas y en este caso se llaman las diferencias 0-esima y primera, respectivamente.

- La segunda diferencia dividida se define como:

  $$
  f\left[x_{i}, x_{i+1}, x_{i+2}\right]=\frac{f\left[x_{i+1}, x_{i+2}\right]-f\left[x_{i}, x_{i+1}\right]}{x_{i+2}-x_{i}}
  $$

- Se puede generalizar para producir la $k-$esima diferencia dividida así:

  $$
  f\left[x_{i}, x_{i+1}, \ldots, x_{i+k-1}, x_{i+k}\right]=\frac{f\left[x_{i+1}, x_{i+2}, \ldots, x_{i+k}\right]-f\left[x_{i}, x_{i+1}, \ldots, x_{i+k-1}\right]}{x_{i+k}-x_{i}}
  $$

- Con estas definiciones es posible probar que el polinomio interpolante de Lagrange es:

  $$
  \begin{aligned}
P_{n}(x)=& f\left[x_{0}\right]+f\left[x_{0}, x_{1}\right]\left(x-x_{0}\right)+a_{2}\left(x-x_{0}\right)\left(x-x_{1}\right) \\
&+\cdots+a_{n}\left(x-x_{0}\right)\left(x-x_{1}\right) \cdots\left(x-x_{n-1}\right)
\end{aligned}
  $$

- Generalizando queda:

  $$
  P_{n}(x)=f\left[x_{0}\right]+\sum_{k=1}^{n} f\left[x_{0}, x_{1}, \ldots, x_{k}\right]\left(x-x_{0}\right) \cdots\left(x-x_{k-1}\right)
  $$

- Las diferencias divididas se pueden calcular usando un esquema tabular como se muestra en la figura:

 <center>
  <img src="https://raw.githubusercontent.com/JorgeZuluaga/NotasMetodosNumericos/master/figuras/diferencias-divididas.png" width=800>
  </center>

- Nótese que el valor de los coeficientes es igual a la diagonal de la matriz.

### Algoritmo de diferencias divididas

- En la imagen abajo se muestra el algoritmo para calcular los elementos de la tabla de diferencias divididas: 

  <center>
  <img src="https://raw.githubusercontent.com/JorgeZuluaga/NotasMetodosNumericos/master/figuras/algoritmo-diferencias-divididas.png" width=600>
  </center>

- Una implementación en Python de este algoritmo se muestra abajo:


In [ ]:
def diferencias_divididas(xs,fs):
  import sympy as sp
  import numpy as np
  from prettytable import PrettyTable
  #Orden del polinomio
  n=len(xs)-1
  #Crea la matriz de los valores
  F=np.ones((n+1,n+1))*np.nan
  #Inicia la primera columna
  F[0,0]=fs[0]
  for i in range(1,n+1):
    F[i,0]=fs[i]
  #Llena la tabla
  for i in range(1,n+1):
    for j in range(1,i+1):
      F[i,j]=(F[i,j-1]-F[i-1,j-1])/(xs[i]-xs[i-j])
  #Crea el polinomio
  x=sp.symbols("x")
  Pn=F[0,0]
  for k in range(1,n+1):
    factor=1
    for i in range(k):
      factor*=(x-xs[i])
    Pn+=F[k,k]*factor
  Pn=sp.expand(Pn)
  #Crea la función
  Pfun=sp.lambdify(x,Pn)
  #Vamos a crear la tabla
  N=len(xs)
  head=["x"]
  for i in range(N):
    head+=[f"f[{''.join((',',)*i)}]"]
  tabla=PrettyTable(head)
  for i in range(N):
    row=[f"{xs[i]}",f"{F[i,0]}"]
    for j in range(1,N):
      row+=[f"{F[i][j]:.6f}"] if not np.isnan(F[i][j]) else ["-"]
    tabla.add_row(row)
  return F,tabla,Pn,Pfun

- **Ejercicio 1, literal a, Sección 3.3**. 
  - Encontrar el valor de la función f(8.4) si f(8.1)=16.94410, f(8.3)=17.56492, f(8.6)=18.50515, f(8.7)=18.82091, usando polinomios interpolantes de grado 1, 2 y 3.  

- **Solución**:

  - Vamos a resolverlo para el caso de polinomio de grado dos.

  - Escogemos valores que incluyan $x=8.4$ así: $x_0=8.3$, $x_1=8.6$, $x_2=8.7$ y $f(x_0)=17.56492$, $f(x_1)=18.50515$, $f(x_2)=18.82091$.

  - Podemos con estos valores llamar a la rutina:

In [ ]:
xs=[8.3,8.6,8.7]
fs=[17.56492,18.50515,18.82091]
F,tabla,Pn,Pfun=diferencias_divididas(xs,fs)

Esta es la tabla resultante:

In [ ]:
print(tabla)

+-----+----------+----------+----------+
|  x  |   f[]    |   f[,]   |  f[,,]   |
+-----+----------+----------+----------+
| 8.3 | 17.56492 |    -     |    -     |
| 8.6 | 18.50515 | 3.134100 |    -     |
| 8.7 | 18.82091 | 3.157600 | 0.058750 |
+-----+----------+----------+----------+


- **Solución (cont.)**:

  - Para escribir el polinomio tomamos los elementos de la diagonal así:

   $$
  \begin{aligned}
  P_{3}&=f[x_0]+f[x_0,x_1](x-x_0)+f[x_0,x_1,x_2](x-x_0)(x-x_1)\\
  &=17.56492+3.134100(x-8.3)+0.058750(x-8.3)(x-8.6)
  \end{aligned}
  $$
  - Teniendo la matriz `F`, el polinomio analítico `Pn` y la función numérica `Pfun` podemos encontrar la solución:

In [ ]:
Pn

0.0587500000000265*x**2 + 2.14122499999956*x - 4.25453499999819

In [ ]:
Pfun(8.4)

17.877154999999984

## Interpolación de Hermite

- La interpolación de Hermite busca resolver el problema que habíamos visto al interpolar datos reales, en los que la función oscilaba de forma "salvaje".

- La interpolación de Hermite es un método para encontrar un polinomio que pase por los puntos de un conjunto de datos pero que también tenga primera derivada igual a la derivada del polinomio.

- El polinomio que cumple esa condición se llama polinomio de Hermite y es de grado $2n+1$:

  $$
  H_{2 n+1}(x)=\sum_{j=0}^{n} f\left(x_{j}\right) H_{n, j}(x)+\sum_{j=0}^{n} f^{\prime}\left(x_{j}\right) \hat{H}_{n, j}(x)
  $$

- Los coeficientes del polinomio se calculan así:

  $$
  H_{n, j}(x)=\left[1-2\left(x-x_{j}\right) L_{n, j}^{\prime}\left(x_{j}\right)\right] L_{n, j}^{2}(x)
  $$

  y 

  $$
  \hat{H}_{n, j}(x)=\left(x-x_{j}\right) L_{n, j}^{2}(x)
  $$

- Como el cálculo derivando los coeficientes de los polinomios de Lagrange es tan laborioso, existe una manera de calcular los polinomios de Hermite usando diferencias divididas:

  $$
  H_{2 n+1}(x)=f\left[z_{0}\right]+\sum_{k=1}^{2 n+1} f\left[z_{0}, \ldots, z_{k}\right]\left(x-z_{0}\right)\left(x-z_{1}\right) \cdots\left(x-z_{k-1}\right)
  $$

  donde introducimos un conjunto de vaiables nuevas $z_{2 i}=z_{2 i+1}=x_{i}$ y $f\left[z_{2 i}, z_{2 i+1}\right]=f^{\prime}\left(z_{2 i}\right)=f^{\prime}\left(x_{i}\right)$.

- El método se ilustra en la tabla a continuación:

<center>
  <img src="https://raw.githubusercontent.com/JorgeZuluaga/NotasMetodosNumericos/master/figuras/tabla-hermite.png" width=600>
  </center>



### Algoritmo de la interpolación de Hermite

- Este es el algoritmo de la interpolación de Hermite:

<center>
  <img src="https://raw.githubusercontent.com/JorgeZuluaga/NotasMetodosNumericos/master/figuras/algoritmo-hermite-1.png" width=600>
  </center>
<center>
  <img src="https://raw.githubusercontent.com/JorgeZuluaga/NotasMetodosNumericos/master/figuras/algoritmo-hermite-2.png" width=600>
  </center>

- Esta es la implementación en Python del método:



In [ ]:
def interpolacion_hermite(xs,fs,fps):
  import sympy as sp
  import numpy as np
  from prettytable import PrettyTable
  #Orden del polinomio
  n=len(xs)-1
  #Crea la matriz de los valores
  zs=np.zeros(2*n+2)
  Qs=np.ones((2*n+2,2*n+2))*np.nan
  #Inicia la primera columna
  for i in range(n+1):
    zs[2*i]=xs[i]
    zs[2*i+1]=xs[i]
    Qs[2*i,0]=fs[i]
    Qs[2*i+1,0]=fs[i]
    Qs[2*i+1,1]=fps[i]
    if i!=0:
      Qs[2*i,1]=(Qs[2*i,0]-Qs[2*i-1,0])/(zs[2*i]-zs[2*i-1])
  #Calcula la matriz
  for i in range(2,2*n+2):
    for j in range(2,i+1):
      Qs[i,j]=(Qs[i,j-1]-Qs[i-1,j-1])/(zs[i]-zs[i-j])
  #Crea el polinomio
  x=sp.symbols("x")
  Pn=0
  for k in range(0,2*n+2):
    factor=1
    for i in range(k):
      factor*=(x-zs[i])
    Pn+=Qs[k,k]*factor
  Pn=sp.expand(Pn)
  #Crea la función
  Pfun=sp.lambdify(x,Pn)
  #Vamos a crear la tabla
  N=len(zs)
  head=["z"]
  for i in range(N):
    head+=[f"f[{''.join((',',)*i)}]"]
  tabla=PrettyTable(head)
  for i in range(N):
    row=[f"{zs[i]}",f"{Qs[i,0]}"]
    for j in range(1,N):
      row+=[f"{Qs[i][j]:.6f}"] if not np.isnan(Qs[i][j]) else ["-"]
    tabla.add_row(row)
  return Qs,tabla,Pn,Pfun

- **Ejercicio 1, literal b, Sección 3.4**. 
  - Construir un polinimio de aproximación de Hermite para los siguientes datos:

|$x$|$f(x)$|$f'(x)$|
|-|-|-|
|0.8|0.22363362|2.1691753|
|1.0|0.65809197|2.0466965|

- **Solución**:

  - Llamamos la rutina con los datos provistos:

In [ ]:
xs=[0.8,1.0]
fs=[0.22363362,0.65809197]
fps=[2.1691753,2.0466965]
Qs,tabla,Pn,Pfun=interpolacion_hermite(xs,fs,fps)

Esta es la tabla resultante:

In [ ]:
print(tabla)

+-----+------------+----------+-----------+-----------+
|  z  |    f[]     |   f[,]   |   f[,,]   |   f[,,,]  |
+-----+------------+----------+-----------+-----------+
| 0.8 | 0.22363362 |    -     |     -     |     -     |
| 0.8 | 0.22363362 | 2.169175 |     -     |     -     |
| 1.0 | 0.65809197 | 2.172292 |  0.015582 |     -     |
| 1.0 | 0.65809197 | 2.046696 | -0.627976 | -3.217793 |
+-----+------------+----------+-----------+-----------+


De modo que el polinomio de Hermite sería:

$$
\begin{aligned}
H_{3}&=f[z_0]+f[z_0,z_1](x-z_0)+f[z_0,z_1,z_2](x-z_0)(x-z_1)+f[z_0,z_1,z_2,z_3](x-z_0)(x-z_1)(x-z_2)\\
&= 0.22363362+2.169175(x-0.8)+0.015582(x-0.8)(x-0.8)-3.217793(x-0.8)(x-0.8)(x-0.8)
\end{aligned}
$$

Que simplificado (nos lo da la rutina) es:

In [ ]:
Pn

-3.21779250000002*x**3 + 8.38184275000005*x**2 - 5.06361150000004*x + 0.557653220000012

Aunque el problema no lo pidió, podemos evaluarlo en algún punto, por ejemplo $x=0.9$:


## Continuará...

## Ejercicios

- Resolver los ejercicios:

  - De la sección 3.3: ejercicio 2, literal b.
  - De la sección 3.4 (a mano): ejercicio 8, literal a.
  - De la sección 3.4 (con rutinas): ejercicio 9, literal a.

-----
*Fin*